<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/6_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 1.  select all (根據filter(最終展示的) 勾選)

In [ ]:
import dash
from dash import Dash, dash_table, html, dcc, Output, Input, State
import pandas as pd

# 假資料
df = pd.DataFrame({
    "姓名": ["Alice", "Bob", "Charlie", "David", 'Cindy'],
    "年齡": ['25', '30', '35', '40', '30'],
    "城市": ["台北", "高雄", "台中", "台南", "台北"]
})

app = Dash(__name__)

app.layout = html.Div([
    # ✅ Select All 勾選框
    dcc.Checklist(
        id='select-all',
        options=[{'label': 'Select All (filtered)', 'value': 'all'}],
        value=[],
        style={'marginBottom': '10px'}
    ),

    # ✅ DataTable with filter
    dash_table.DataTable(
        id='my-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        row_selectable="multi",
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'}
    )
])

# ✅ Callback：根據 filter 後的資料決定選取哪些列
@app.callback(
    Output('my-table', 'selected_rows'),
    Input('select-all', 'value'),
    State('my-table', 'derived_virtual_data'),  # DataTable 篩選後剩下的資料
    State('my-table', 'data')
)
def select_filtered_rows(select_all_value, filtered_data, full_data):
    if 'all' in select_all_value and filtered_data is not None:
        # 對應回原始資料中的 index (回推哪些 row 在篩選後仍存在。)
        return [i for i, row in enumerate(full_data) if row in filtered_data]
    return []

if __name__ == '__main__':
    app.run(debug=True)


### 2. select all (根據 filter) + 勾選的部分 粉色底色

In [ ]:
import dash
from dash import Dash, dash_table, html, dcc, Output, Input, State
import pandas as pd

# 假資料 + 原始索引欄
df = pd.DataFrame({
    "姓名": ["Alice", "Bob", "Charlie", "David", 'Cindy'],
    "年齡": ['25', '30', '35', '40', '30'],
    "城市": ["台北", "高雄", "台中", "台南", "台北"]
})
### 每個 row 需要一個'獨立index'
df['_index'] = df.index

app = Dash(__name__)

app.layout = html.Div([
    dcc.Checklist(
        id='select-all',
        options=[{'label': 'Select All (filtered)', 'value': 'all'}],
        value=[],
        style={'marginBottom': '10px'}
    ),
    dash_table.DataTable(
        id='my-table',
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        filter_action="native",
        row_selectable="multi",
        selected_rows=[],
        style_table={'overflowX': 'auto'},
        style_cell={'textAlign': 'left'},
        style_data_conditional=[]
    )
])

# ✅ 控制勾選：根據篩選後資料決定 selected_rows（回傳原始 index）
# 1. 取得「篩選後」資料（來自 derived_virtual_data)
# 2. 找出這些資料在原始資料中的 index
# 3. 回傳這些 _index，正確設定 selected_row (原始資料的)
@app.callback(
    Output('my-table', 'selected_rows'),
    Input('select-all', 'value'),
    State('my-table', 'derived_virtual_data'),
    State('my-table', 'data')
)
def select_filtered_rows(select_all_value, filtered_data, full_data):
    if 'all' in select_all_value and filtered_data is not None:
        return [row['_index'] for row in filtered_data]
    return []

# ✅ 底色樣式：根據篩選資料中的原始 index 與勾選狀態設定底色
# (透過Input: derived_virtual_data，讓 Dash 只要一變動就觸發底色更新 callback，確保樣式正確。)
@app.callback(
    Output('my-table', 'style_data_conditional'),
    Input('my-table', 'derived_virtual_data'),
    Input('my-table', 'selected_rows')
)
def update_row_style(filtered_data, selected_rows):
    if not selected_rows or not filtered_data:
        return []

    return [
        {
            'if': {'row_index': i},
            'backgroundColor': '#ffe6f0',
            'border': '1px solid #ff99cc'
        }
        for i, row in enumerate(filtered_data)
        if row.get('_index') in selected_rows
    ]

if __name__ == '__main__':
    app.run(debug=True)